In [87]:
import urllib.request
import datetime
import os
import pandas as pd


In [91]:
def download_vhi_data():
    for i in range(1, 26):  # Для 25 областей України
        # Формуємо URL для запиту
        url = f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={i}&year1=1981&year2=2024&type=Mean"
        print(f"Крок {i}: Формую URL для області {i}: {url}")
        
        # Отримуємо поточну дату і час
        data_sign = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
        filename = f"VHI_{i}_{data_sign}.csv"
        print(f"Крок {i}: Формую ім'я файлу: {filename}")
        
        # Перевіряємо, чи існує файл
        if not os.path.exists(filename):
            print(f"Крок {i}: Файл не існує. Завантажую дані з URL...")
            # Якщо файл не існує, завантажуємо дані
            try:
                response = urllib.request.urlopen(url)
                my_data = response.read()
                print(f"Крок {i}: Дані успішно завантажені.")
                
                # Зберігаємо файл
                with open(filename, "wb") as file:
                    file.write(my_data)
                print(f"Крок {i}: Файл збережено: {filename}")
            except Exception as e:
                print(f"Крок {i}: Помилка при завантаженні даних: {e}")
        else:
            print(f"Крок {i}: Файл {filename} вже існує.")
            
download_vhi_data()



Крок 1: Формую URL для області 1: https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID=1&year1=1981&year2=2024&type=Mean
Крок 1: Формую ім'я файлу: VHI_1_20250403141303.csv
Крок 1: Файл не існує. Завантажую дані з URL...
Крок 1: Дані успішно завантажені.
Крок 1: Файл збережено: VHI_1_20250403141303.csv
Крок 2: Формую URL для області 2: https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID=2&year1=1981&year2=2024&type=Mean
Крок 2: Формую ім'я файлу: VHI_2_20250403141311.csv
Крок 2: Файл не існує. Завантажую дані з URL...
Крок 2: Дані успішно завантажені.
Крок 2: Файл збережено: VHI_2_20250403141311.csv
Крок 3: Формую URL для області 3: https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID=3&year1=1981&year2=2024&type=Mean
Крок 3: Формую ім'я файлу: VHI_3_20250403141314.csv
Крок 3: Файл не існує. Завантажую дані з URL...
Крок 3: Дані успішно завантажені.
Крок 3: Файл збережено: VHI_3

Цей код проходить по всіх областях (з 1 до 28), отримує дані та зберігає їх у файли з унікальною назвою   

In [78]:

def load_data_to_dataframe(files):
    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
    
    files = [file for file in os.listdir() if file.endswith(".csv")]
    df_list = []
    
    for file in files:
        df = pd.read_csv(file, header=1, names=headers) 
        df = df.drop(columns=['empty'], errors='ignore')
        df = df.drop(df[df['VHI'] == -1].index)  
        df_list.append(df)
    
    df = pd.concat(df_list, ignore_index=True)
    print(df.head())

Читаємо всі файли у папці та додаємо заголовки. Виводисо перші 5 рядків для перевірки

In [79]:
def update_area_indexes(df):
    index_map = {
        1: 'Вінницька', 2: 'Волинська', 3: 'Дніпропетровська', 4: 'Донецька',
        5: 'Житомирська', 6: 'Закарпатська', 7: 'Запорізька', 8: 'Івано-Франківська',
        9: 'Київська', 10: 'Кіровоградська', 11: 'Луганська', 12: 'Львівська',
        13: 'Миколаївська', 14: 'Одеська', 15: 'Полтавська', 16: 'Рівенська',
        17: 'Сумська', 18: 'Тернопільська', 19: 'Харківська', 20: 'Херсонська',
        21: 'Хмельницька', 22: 'Черкаська', 23: 'Чернівецька', 24: 'Чернігівська',
        25: 'Республіка Крим'
    }
    
    df["area"].replace(index_map, inplace=True)
    
    return df


Оскільки в оригінальних файлах область позначена лише номером, потрібно додати відповідність номеру та назві області

In [80]:
def get_vhi_for_year(df, province_index, year):
    return df[(df["area"] == province_index) & (df["Year"] == year)]['VHI']

def get_extremes(df, province_index, year):
    vhi_data = df[(df["area"] == province_index) & (df["Year"] == year)]['VHI']
    min_v = vhi_data.min()
    max_v = vhi_data.max()
    return min_v, max_v

def get_droughts(df):
    return df[(df['VHI'] <= 15) & (df['VHI'] != -1)]

def get_vhi_for_year_range(df, province_index, start_year, end_year):
    return df[(df["area"] == province_index) & (df["Year"] >= start_year) & (df["Year"] <= end_year)]['VHI']

def get_extreme_droughts(df, percentage=20):
    years = df['Year'].unique()
    extreme_years = []
    for year in years:
        drought_areas = df[(df['Year'] == year) & (df['VHI'] <= 15)]['area'].nunique()
        if drought_areas / 25 * 100 >= percentage:
            extreme_years.append(year)
    
    return extreme_years



Знаходимо мінімальні та максимальні значення VHI

In [81]:
def print_drought_analysis(df, extreme_years):
    print("Екстремальні посухи відбулися в наступні роки:")
    for year in extreme_years:
        drought_areas = df[(df['Year'] == year) & (df['VHI'] <= 15)]
        print(f"Рік: {year}")
        print(f"Області: {', '.join(drought_areas['area'].unique())}")
        print(f"Значення VHI: {drought_areas['VHI'].tolist()}")
